<a href="https://colab.research.google.com/github/Gaukhar-ai/for_my_Thinkful_work/blob/master/NLP_Feature_Engineering_I_text_to_numbers_train_test_split_models_scores_GBC_RF_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import nltk
nltk.download('gutenberg')
#!python -m spacy download en

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [2]:
import numpy as np
import pandas as pd
import sklearn
import spacy
import re
from nltk.corpus import gutenberg
import nltk
import warnings
warnings.filterwarnings('ignore')

In [3]:
#utility function for standard text cleaning

In [4]:
def text_cleaner(text):

  #visual inspection idenifies a form of punctuation that spaCy doesn't recognize:
  #the double dash '--'. better get rid of it now.

  text = re.sub(r'--', ' ', text)
  text = re.sub('[\[].*?[\]]', '', text)
  text = re.sub(r'(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b', ' ', text)
  text = ' '.join(text.split())
  return text

In [5]:
#load the clean data. Removing the chapter indicators then text_cleaner function

In [6]:
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER.*', '', alice)

alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [7]:
#split the texts into sentences with spaCy 

In [8]:
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [9]:
#iterate over the parsed docs after calling .sents = can iterate using list comprehension

In [10]:
#group into sentences
alice_sents = [[sent, 'Carroll'] for sent in alice_doc.sents]
persuasion_sents = [[sent, 'Austen'] for sent in persuasion_doc.sents]

#combine the sentences from the two novels into one DF
sentences = pd.DataFrame(alice_sents+persuasion_sents, columns=['text', 'author'])
sentences.head()

,text,author
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [11]:
#remove stopwords, punctuation marks, convert tokens to lemmas or stems)

for i, sentence in enumerate(sentences['text']):
  sentences.loc[i, 'text'] = " ".join(
      [token.lemma_ for token in sentence if not token.is_punct and not token.is_stop])

In [12]:
#converting a text into numbers with BoW approach, use CountVectorizer from scikit-learn

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word')
X = vectorizer.fit_transform(sentences['text'])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentences = pd.concat([bow_df, sentences[['text', 'author']]], axis=1)

In [15]:
sentences.head()

,1st,29th,abbreviation,abdication,abide,ability,able,abode,abominable,abominate,abroad,absence,absent,absolute,absolutely,abstraction,absurd,absurdity,abundance,abuse,abydos,accent,accept,acceptable,acceptance,accession,accident,accidental,accidentally,accommodate,accommodation,accompany,accomplish,accomplished,accomplishment,accord,accordingly,accost,account,accounting,...,wrapt,wreck,wretched,wretchedly,wretchedness,wriggle,wrinkle,wrist,write,writhing,writing,wrong,wrought,yard,yarmouth,yawn,ye,year,yearly,yell,yelp,yeoman,yer,yes,yesterday,yestermorn,yield,yielding,you,young,younker,youth,youthful,zeal,zealand,zealous,zealously,zigzag,text,author
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Alice begin tired sit sister bank have twice p...,Carroll
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,consider mind hot day feel sleepy stupid pleas...,Carroll
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,remarkable Alice think way hear Rabbit oh dear,Carroll
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,oh dear,Carroll
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,shall late,Carroll


In [16]:
#BoW in action:

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [18]:
#need to find who's the author Y
Y = sentences['author']
X = np.array(sentences.drop(['text', 'author'], 1))

In [19]:
#split the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=55)

In [20]:
#models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

In [21]:
lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [31]:
print('\n----------------Logistic Regression Scores ---------------------')
print('Training set score:', lr.score(X_train, y_train))
print('Test set score:', lr.score(X_test, y_test))


print('\n----------------Random Forest Classifier  ---------------------')
print('Training set score:', rfc.score(X_train, y_train))
print('Test set score:', rfc.score(X_test, y_test))


print('\n----------------  GradientBoostingClassifier  ---------------------')
print('Training set score:', gbc.score(X_train, y_train))
print('Test set score:', gbc.score(X_test, y_test))


----------------Logistic Regression Scores ---------------------
Training set score: 0.9211201368106028
Test set score: 0.8666666666666667

----------------Random Forest Classifier  ---------------------
Training set score: 0.9675074818298418
Test set score: 0.847008547008547

----------------  GradientBoostingClassifier  ---------------------
Training set score: 0.8264215476699445
Test set score: 0.8153846153846154


In [32]:
#n-grams = 2-gram, 3-gram. One word means several things, to distinguish what the meaning is it combined into 2, 3 words
# she labored in vain = she labored, labored in, in vain
#she was so vain, her bathroom mirror was covered in lip prints = 3-grams: she was so, was so vain, so vain her, vain her bathroom.etc

In [34]:
#use 2-grams
vectorizer = CountVectorizer(analyzer='word', ngram_range=(2,2))
X = vectorizer.fit_transform(sentences['text'])
bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
sentence = pd.concat([bow_df, sentences[['text', 'author']]], axis=1)
sentences.head()

,1st,29th,abbreviation,abdication,abide,ability,able,abode,abominable,abominate,abroad,absence,absent,absolute,absolutely,abstraction,absurd,absurdity,abundance,abuse,abydos,accent,accept,acceptable,acceptance,accession,accident,accidental,accidentally,accommodate,accommodation,accompany,accomplish,accomplished,accomplishment,accord,accordingly,accost,account,accounting,...,wrapt,wreck,wretched,wretchedly,wretchedness,wriggle,wrinkle,wrist,write,writhing,writing,wrong,wrought,yard,yarmouth,yawn,ye,year,yearly,yell,yelp,yeoman,yer,yes,yesterday,yestermorn,yield,yielding,you,young,younker,youth,youthful,zeal,zealand,zealous,zealously,zigzag,text,author
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Alice begin tired sit sister bank have twice p...,Carroll
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,consider mind hot day feel sleepy stupid pleas...,Carroll
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,remarkable Alice think way hear Rabbit oh dear,Carroll
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,oh dear,Carroll
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,shall late,Carroll


In [37]:
# 2-gram as feature:

Y = sentences['author']
X = np.array(sentences.drop(['text', 'author'], 1))

#split the data to train test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=55)

#models
lr = LogisticRegression()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

lr.fit(X_train, y_train)
rfc.fit(X_train, y_train)
gbc.fit(X_train, y_train)

print('---------------Logisitic Regression scores --------------------')
print('Training set score:', lr.score(X_train, y_train))
print('Test set score:', lr.score(X_test, y_test))

print('\n----------------Random Forest score -------------------------')
print('Training set score:', rfc.score(X_train, y_train))
print('Test set score:', rfc.score(X_test, y_test))

print('---------------Gradient Boosting scores --------------------')
print('Training set score:', gbc.score(X_train, y_train))
print('Test set score:', gbc.score(X_test, y_test))

---------------Logisitic Regression scores --------------------
Training set score: 0.9211201368106028
Test set score: 0.8666666666666667

----------------Random Forest score -------------------------
Training set score: 0.9675074818298418
Test set score: 0.847008547008547
---------------Gradient Boosting scores --------------------
Training set score: 0.8264215476699445
Test set score: 0.8153846153846154
